<a href="https://colab.research.google.com/github/chanhyeong00/machine_learning_study/blob/main/pytorch/basic_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 퍼셉트론(perceptron)이란

인공 신경망의 한 종류로, 출력이 0 또는 1인 작업을 의미하는 이진 분류 작업에 사용되는 간단한 모델이다.

퍼셉트론은 신경세포(Neuron)가 신호를 전달하는 구조와 유사한 방식으로 구현됐다.

퍼셉트론은 TLU(Thershold Logic Unit) 형태를 기반으로 하며, 계단 함수(step function)를 적용해 결과를 반환한다.

### 단층 퍼셉트론(Single Layer Perceptron)
하나의 계층을 갖는 모델을 의미한다.

- 단층 퍼셉트론 한계
  - AND, OR, NAND 게이트와 같은 구조를 갖는 모델은 쉽게 구현 가능하다.
  - XOR 게이트처럼 하나의 기울기로 표현하기 어려운 구조에서는 단층 퍼셉트론을 적용하기 어렵다.


#### 구조

In [1]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

In [2]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.y = df.iloc[:, 2].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer = nn.Sequential(
            nn.Linear(2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer(x)
        return x

In [3]:
train_dataset = CustomDataset("perceptron.csv")
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)

perceptron.csv 데이터셋은 XOR 문제이다.

In [4]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [5]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

<ipython-input-2-b746038c3bc6>:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  x = torch.FloatTensor([self.x1[index], self.x2[index]])
<ipython-input-2-b746038c3bc6>:11: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])


Epoch : 1000, Cost : 0.692
Epoch : 2000, Cost : 0.692
Epoch : 3000, Cost : 0.692
Epoch : 4000, Cost : 0.692
Epoch : 5000, Cost : 0.692
Epoch : 6000, Cost : 0.692
Epoch : 7000, Cost : 0.692
Epoch : 8000, Cost : 0.692
Epoch : 9000, Cost : 0.692
Epoch : 10000, Cost : 0.692


오차가 꽤나 크고 줄지 않는 것으로 보아 하나의 계층으로는 XOR 문제를 해결할 수 없는 거 같다.

In [6]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor([
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1]
    ]).to(device)
    outputs = model(inputs)

    print("---------")
    print(outputs)
    print(outputs <= 0.5)

---------
tensor([[0.4677],
        [0.5000],
        [0.5035],
        [0.5358]], device='cuda:0')
tensor([[ True],
        [ True],
        [False],
        [False]], device='cuda:0')


모델에 값을 입력했을 때도 출력값이 0.5 내외로 출력돼 학습이 정상적으로 진행되지 않는 것을 볼 수 있다.

### 다층 퍼셉트론(Multi-Layer Perceptron, **MLP**)

은닉층이 하나 이상인 퍼셉트론 구조를 말한다.

은닉층을 2개 이상 연결하면 심층신경망(Deep Neural Network, DNN) 이라고 부른다.

**학습 방법**

1. 입력층부터 출력층까지 순전파(forward propagation) 진행
2. 출력값(예측값)과 실젯값으로 오차 계산
3. 오차를 퍼셉트론의 역방향으로 보내면서 입력된 노드의 기여도 측정
    - 손실 함수를 편미분해 기울기 계산
    - 연쇄 법칙(Chain Rule)을 통해 기울기 계산
4. 입력층에 도달할 때까지 노드의 기여도 측정
5. 모든 가중치에 최적화 알고리즘 수행

#### 구조

2개 은닉층 사용

In [7]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Linear(2, 2),
            nn.Sigmoid()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [9]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

<ipython-input-2-b746038c3bc6>:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  x = torch.FloatTensor([self.x1[index], self.x2[index]])
<ipython-input-2-b746038c3bc6>:11: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])


Epoch : 1000, Cost : 0.693
Epoch : 2000, Cost : 0.691
Epoch : 3000, Cost : 0.611
Epoch : 4000, Cost : 0.473
Epoch : 5000, Cost : 0.172
Epoch : 6000, Cost : 0.056
Epoch : 7000, Cost : 0.032
Epoch : 8000, Cost : 0.022
Epoch : 9000, Cost : 0.016
Epoch : 10000, Cost : 0.013


In [10]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor([
        [0, 0],
        [0, 1],
        [1, 0],
        [1, 1]
    ]).to(device)
    outputs = model(inputs)

    print("---------")
    print(outputs)
    print(outputs <= 0.5)

---------
tensor([[0.0098],
        [0.9858],
        [0.9858],
        [0.0142]], device='cuda:0')
tensor([[ True],
        [False],
        [False],
        [ True]], device='cuda:0')


보면 XOR 문제를 단층으론 손실이 매우 크지만 다층 퍼셉트론으론 매우 낮은 손실을 얻어내는 것을 볼 수 있다.

실제 결과와 예측값도 동일한 것을 볼 수 있다.

퍼셉트론은 많은 머신러닝 애플리케이션에서 사용된다. 특히 이진 분류 작업에서 여전히 사용되는 간단하고 효율적인 모델이다.

**하지만 데이터의 복잡한 패턴을 학습할 수 없으며, 선형으로 분리되지 않는 데이터를 분류할 수 없는 등 몇 가지 제한 사항이 있다.**

이러한 제한으로 보다 복잡한 작업에 더 적합한 고급 신경망 모델이 개발됐다.